# Análisis Exploratorio de la respuesta de AT&T a solicitud ARCO

En este _notebook_ se hace un análisis de la información proporcionada por _AT&T México_ como respuesta a una solicitud [ARCO](http://www.infoem.org.mx/src/htm/queEsArco.html) realizada por José Luis Soto Galindo. Pues conforme a la ley, la empresa telefónica está obligada a proporcionar al usuario todos los datos personales que almacene, pues él es el titular de ésta información.

La información que están obligados a proporcionar está definida según el reglamento como:
![reglamento](rawdata/descripcion_campos_obligados_entregar_telefonicas.png)

_AT&T_ entregó la información en un [archivo PDF](rawdata/documental-05-02-documento-pdf-responsable-23-agosto-2016.pdf) en formato de una tabla con los campos definidos según su  [respuesta](rawdata/respuesta_inai_att_soto_galindo.pdf) como:
![campos parte 1](rawdata/campos1.png)
![campos parte 2](rawdata/campos2.png)


Se plantea a petición del dueño de ésta información saber que inferencias se pueden hacer de su vida personal (viajes, contactos frecuentes, hábitos) con ésta información en manos de terceros, pues de acuerdo a un [reporte de solicitudes del IFT](rawdata/reportes_solicitudes_ift_2016.xlsx) hay solicitudes de éstos datos que no fueron realizadas por instituciones judiciales, lo que genera la preocupación de cual es el uso final que le dan a éstos datos.

- Uso de DATOS
- Empresas telefónicas
- Días de descanso (contactos discriminados entre semana y fin de semana)

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly
from dateutil.parser import parse
plotly.offline.init_notebook_mode(connected=True)
import cufflinks as cf

In [361]:
att = pd.read_csv("rawdata/tabula-documental-05-02-documento-pdf-responsable-23-agosto-2016.csv")
#att.loc[att["Ciudad"] == "90150001","Ciudad"]="CIUDAD DE MEXICO CUAUHTEMOC"
att["timestamp"] = (att["Fecha"] + " " + att["Hora"]).map(parse)
att.sort_values(by="timestamp",inplace=True)

In [362]:
conteosEntranteSaliente = att[att["Evento"].isin(["LLAMADA ENTRANTE","LLAMADA SALIENTE","SMS"])][["Número","Evento"]].groupby(["Evento","Número"]).size()
conteosEntranteSaliente = conteosEntranteSaliente.swaplevel().unstack().fillna(0)
conteosEntranteSaliente["TOTAL"] = conteosEntranteSaliente["LLAMADA ENTRANTE"] + conteosEntranteSaliente["LLAMADA SALIENTE"] + conteosEntranteSaliente["SMS"]
conteosEntranteSaliente = conteosEntranteSaliente[conteosEntranteSaliente["TOTAL"] > 30].sort_values(by="TOTAL")

In [363]:
conteosEntranteSaliente.index
conteosIndex = conteosEntranteSaliente.index.tolist()
#Nombres extraídos de Facebook
conteosIndex[conteosIndex.index('525515905964')] = 'Patricia Amador'
conteosIndex[conteosIndex.index('525554377955')] = 'Mario Calixto'
conteosIndex[conteosIndex.index('523333688344')] = 'Monica Soto Galindo'
conteosIndex[conteosIndex.index('523318118121')] = 'Paola Soto Galindo'
conteosIndex[conteosIndex.index('525564167804')] = 'Luis Blisete'
conteosIndex[conteosIndex.index('525527684512')] = 'Sisi Mendoza'
conteosIndex[conteosIndex.index('5561220633')] = 'Arian Ddaszshintté'
conteosIndex[conteosIndex.index('525591832100')] = 'C. N. D. de los Pueblos Indígenas'
conteosEntranteSaliente.index = conteosIndex

## Relaciones personales
Como primera cuestión se plantea saber si se pueden inferir relaciones personales con los registros de operaciones proporcionados. Analizando el número de llamadas entrantes y salientes se obtiene la siguiente gráfica.

In [364]:
title = 'Cantidad de llamadas telefónicas, entrantes y salientes para los números mas frecuentes'
plotly.offline.iplot({"data":[go.Bar(y=conteosEntranteSaliente.index,x=conteosEntranteSaliente["LLAMADA ENTRANTE"],orientation="h",name="Llamadas Entrantes"),go.Bar(y=conteosEntranteSaliente.index,x=conteosEntranteSaliente["LLAMADA SALIENTE"],orientation="h",name="Llamadas Salientes"),go.Bar(y=conteosEntranteSaliente.index,x=conteosEntranteSaliente["SMS"],orientation="h",name="SMS")],"layout":{'barmode':'stack','title':title,'margin':{'l':220},'xaxis':{'title':'Cantidad de llamadas'},'yaxis':{'type':'category','title':'Número telefónico'}}})

De la cual se puede inferir que _Patricia Amador_, _Mario Calixto_ , _Mónica Soto Galindo_, _Paola Soto Galindo_ y _Luis Blisete_ (nombres obtenidos mediante búsqueda de números telefónicos en Facebook) son personas con un contacto frecuente con José Luis Soto Galindo. También se puede inferir que tiene contacto con organizaciones políticas, como la _Comisión Nacional de Desarrollo de los Pueblos Indígenas_.

## Actividad del usuario
¿Se pueden conocer hábitos del usuario por la actividad de su línea telefónica?

In [50]:
title = 'Cantidad de operaciones por fecha'
plotly.offline.iplot({"data":[go.Histogram(x=att["timestamp"])],"layout":{'title':title,"xaxis":{"title":"Fecha"},"yaxis":{"title":"Cantidad de operaciones"}}})

En la gráfica anterior se observa la cantidad de operaciones registradas por cada intervalo de dos semanas, se puede apreciar que la actividad de la línea corresponde principalmente al intervalo que inicia en agosto del 2014 y termina en marzo de 2016, y los puntos más bajos (enero 2015, julio 2015, enero 2016) corresponden a temporadas vacacionales, incluso las últimas dos no registran ninguna operación. Se podría suponer que en el periodo anterior a Julio de 2014 y posterior a Julio de 2016 no era su línea telefónica principal, o que la empresa esta omitiendo información.

In [10]:
horaDeLlamada = att[att["Evento"].isin(['LLAMADA ENTRANTE', 'LLAMADA SALIENTE'])]["timestamp"].apply(lambda x:x.hour)

In [11]:
    title="Conteo de llamadas por hora del día"
    tickvals = np.arange(0,24)
    ticktext =  [str(x) + "h" for x in tickvals]
    plotly.offline.iplot({"data":[go.Histogram(x=horaDeLlamada.values,orientation="v")],"layout":{"title":title,"yaxis":{"title":"Cantidad de llamadas"},"xaxis":{"title":"Hora","tickmode":"array","tickvals":tickvals,"ticktext":ticktext}}})

En la imagen anterior se visualiza la cantidad de llamadas registradas por cada hora del día, de donde se puede inferir el horario de descanso del usuario, así como su hora de mayor actividad.

In [366]:
ciudades = att[np.bitwise_and(att["Ciudad"] != "0",att["Ciudad"] != "90150001")]["Ciudad"].value_counts().sort_values()
plotly.offline.iplot({"data":[go.Bar(y=ciudades.index,x=ciudades.values,orientation="h")],"layout":{'title':'Cantidad de operaciones por ciudad','margin':{'pad':10,'l':400},'yaxis':{'type':'category','title':'Ciudad'},'xaxis':{'title':'Cantidad de operaciones en la ciudad'}}})

Y en ésta imagen se puede identificar la ciudad en la cual el usuario pasa la mayor parte de su tiempo, por lo que se puede inferir que vive y trabaja en la zona formada por la unión de la delegación Cuauhtémoc y la delegación Azcapotzalco, y que viaja regularmente a la ciudad de Guadalajara.

In [372]:
att[att["Evento"] != "DATOS"].groupby("Evento").size().sort_values().iplot(kind="barh",layout={'title':'Cantidad de operaciones por evento','margin':{'l':200,'pad':10},'yaxis':{'title':'Evento'},'xaxis':{'title':'Cantidad de operaciones'}})

Para la gráfica anterior se eliminó la contabilización de el evento _DATOS_, pues éste es mucho más frecuente y regular, y no aporta información sobre el comportamiento del usuario, del cuál se puede inferir que realiza más llamadas de las que recibe, muchas más aún que los mensajes de texto que escribe.

In [374]:
attLocs = att[np.bitwise_not(att["Ciudad"].isin(['90150001','0',np.nan]))].copy()
attLocs["ciudad_ant"] = attLocs.Ciudad.shift()
cambiosCiudad = attLocs[np.bitwise_and(np.bitwise_and(attLocs["ciudad_ant"].notnull(),attLocs["Ciudad"].notnull()),attLocs["Ciudad"] != attLocs["ciudad_ant"])].copy()
cambiosCiudad["cambio"] = cambiosCiudad["Ciudad"]+"-"+cambiosCiudad["ciudad_ant"]
conteosCambios = cambiosCiudad["cambio"].value_counts()
conteosBajos = conteosCambios[conteosCambios.values < 6]
conteosAltos = conteosCambios[conteosCambios.values >= 6]
cambiosBajos = cambiosCiudad[cambiosCiudad["cambio"].isin(conteosBajos.index)]
plotly.offline.iplot({"data":[go.Scatter(x=cambiosBajos["timestamp"],y=cambiosBajos["cambio"],mode="markers")],"layout":{'title':'Cambios de ciudad (realizados menos de 6 veces) contra fecha de su realización','margin':{'l':350},'yaxis':{'tickfont':{'size':8},'type':'category','title':'Número telefónico'}}})

En la gráfica anterior se visualizan los cambios de ciudad que tuvieron lugar en menos de 6 ocasiones (para no observar los viajes ordinarios dentro de la ciudad de México), con ésta información se puede inferir el itinerario de viaje del usuario en el periodo noviembre 2015 - diciembre 2016.

In [376]:
cambiosAltos = cambiosCiudad[cambiosCiudad["cambio"].isin(conteosAltos.index)].copy()
#plotly.offline.iplot({"data":[go.Scatter(x=cambiosBajos["timestamp"],y=cambiosBajos["cambio"],mode="markers")],"layout":{'margin':{'l':290},'yaxis':{'tickfont':{'size':8},'type':'category','title':'Número telefónico'}}})

cambiosAltos["diaCambio"] = cambiosAltos["timestamp"].map(lambda x:x.weekday_name)

cambiosSemanales = cambiosAltos[["cambio","diaCambio"]].groupby(["cambio","diaCambio"]).size().reset_index()

cambiosSemanalesFilt = cambiosSemanales[np.bitwise_and(cambiosSemanales["cambio"] != 'CIUDAD DE MEXICO AZCAPOTZALCO-CIUDAD DE MEXICO CUAUHTEMOC',cambiosSemanales["cambio"] != 'CIUDAD DE MEXICO CUAUHTEMOC-CIUDAD DE MEXICO AZCAPOTZALCO')].sort_values(by=0,ascending=False)

#plotly.offline.iplot({"data":[go.Scatter(y=cambiosSemanalesFilt["diaCambio"],x=cambiosSemanalesFilt["cambio"],mode="markers",marker={"size":8*np.sqrt(cambiosSemanalesFilt[0])})],"layout":{'margin':{'b':200,'r':200},"xaxis":{"tickfont":{"size":8}},"yaxis":{"categoryorder":"array","type":"category","categoryarray":["Sunday","Saturday","Friday","Thursday","Wednesday","Tuesday","Monday"]}}})
plotly.offline.iplot({"data":[go.Scatter(y=cambiosSemanalesFilt["diaCambio"],x=cambiosSemanalesFilt["cambio"],mode="markers",marker={"size":3*cambiosSemanalesFilt[0]})],"layout":{'title':'Cambios de ciudad (realizados mas de 6 veces) contra día de la semana de su realización','margin':{'b':200,'r':200},"xaxis":{"tickfont":{"size":8}},"yaxis":{"categoryorder":"array","type":"category","categoryarray":["Sunday","Saturday","Friday","Thursday","Wednesday","Tuesday","Monday"]}}})

Y por último, se puede observar la cantidad de cambios de ciudad por día de la semana, donde se observa una gran cantidad de viajes en fin de semana a guadalajara, que contrasta con la poca cantidad de viajes a la delegación Miguel Hidalgo desde Azcapotzalco y viceversa.

También se observa una cantidad importante de viajes Azcapotalco-Benito Juárez en viernes, Benito Juárez-Azcapotzalco en sábado y miércoles.